# RDD

RDD (Resilient Distributed Dataset) is a fundamental building block of PySpark which is fault-tolerant, immutable distributed collections of objects. Immutable meaning once you create an RDD you cannot change it. Each record in RDD is divided into logical partitions, which can be computed on different nodes of the cluster.

# Adavantages of RDD

1.Immutablity:
PySpark RDD’s are immutable in nature meaning, once RDDs are created you cannot modify. When we apply transformations on RDD, PySpark creates a new RDD and maintains the RDD Lineage.

2.In-memeory computation:
PySpark loads the data from disk and process in memory and keeps the data in memory, this is the main difference between PySpark and Mapreduce (I/O intensive). In between the transformations, we can also cache/persists the RDD in memory to reuse the previous computations.

3.Fault tolerant:
PySpark RDD’s are immutable in nature meaning, once RDDs are created you cannot modify. When we apply transformations on RDD, PySpark creates a new RDD and maintains the RDD Lineage.

4.Lazy evaluation:
PySpark does not evaluate the RDD transformations as they appear/encountered by Driver instead it keeps the all transformations as it encounters(DAG) and evaluates the all transformation when it sees the first RDD action.

5.Partitioning:
When you create RDD from a data, It by default partitions the elements in a RDD. By default it partitions to the number of cores available.


# Limitations of RDD

PySpark RDDs are not much suitable for applications that make updates to the state store such as storage systems for a web application. For these applications, it is more efficient to use systems that perform traditional update logging and data checkpointing, such as databases. The goal of RDD is to provide an efficient programming model for batch analytics and leave these asynchronous applications.

# Create RDD using sparkContext.parallelize()

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=7241c0a75304d7e5fc318c5dd27d4fcdfa23aefc5912bda6259fa6e05c215f82
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark  = SparkSession.builder.appName('shivashankar').master('local[*]').getOrCreate()

In [3]:
data = list(range(1,11))
data

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [4]:
rdd = spark.sparkContext.parallelize(data)

In [5]:
rdd.glom().collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

In [6]:
rdd.getNumPartitions()

2

In [7]:
#parallelize is used to create a rdd with the hardcoded values.
#for creating a rdd with the external files we have mrthods like
# spark.sparkContext.textFile(path) for creating rdd from textfile
#spark.sparkContext.wholeTextFile(path) function returns a PairRDD with the key being the file path and value being file content


#When we use parallelize() or textFile() or wholeTextFiles() methods of SparkContxt to initiate RDD, it automatically splits the data into partitions based on resource availability.
#when you run it on a laptop it would create partitions as the same number of cores available on your system.

#Set parallelize manually – We can also set a number of partitions manually,
#all, we need is, to pass a number of partitions as the second parameter to these functions
#for example  sparkContext.parallelize([1,2,3,4,56,7,8,9,12,3], 10)


#getNumPartitions() – This a RDD function which returns a number of partitions our dataset split into.



# Create empty RDD using sparkContext.emptyRDD

In [8]:
eRDD = spark.sparkContext.emptyRDD()
eRDD.glom().collect()
#creates empty RDD without partitions

[]

# Creating empty RDD with partition

In [9]:
RDD = spark.sparkContext.parallelize([],10)
RDD.glom().collect()

[[], [], [], [], [], [], [], [], [], []]

Sometimes we may need to repartition the RDD, PySpark provides two ways to repartition
# Repartition and Coalesce

first using repartition() method which shuffles data from all nodes also called full shuffle

second coalesce() method which shuffle data from minimum nodes, for examples if you have data in 4 partitions and doing coalesce(2) moves data from just 2 nodes.

Note that repartition() method is a very expensive operation as it shuffles data from all nodes in a cluster.



In [10]:
RDD.coalesce(2)
RDD.glom().collect()

[[], [], [], [], [], [], [], [], [], []]

In [11]:
RDD.repartition(4)
RDD.glom().collect()

[[], [], [], [], [], [], [], [], [], []]

# PySpark RDD Operations

RDD transformations: Transformations are lazy operations, instead of updating an RDD, these operations return another RDD.

RDD actions :operations that trigger computation and return RDD values.

# RDD TRANSFORMATIONS

# 1.map()
As the name suggests, the .map() transformation maps a value to the elements of an RDD. The .map() transformation takes in an anonymous function and applies this function to each of the elements in the RDD. For example, If we want to add 10 to each of the elements present in RDD, the .map() transformation would come in handy. This operation saves time and goes with the DRY policy.

In [35]:
#map() if we want to perform any transformation on each partition of RDD then we will be using map().
# the function that we give to map will go through every element of RDD.
#map() transformation is used the apply any complex operations like adding a column, updating a column e.t.c.
#the output of map transformations would always have the same number of records as input.

import numpy as np

np.random.seed(100)
data = np.random.randint(1,51,size=15)

rdd = spark.sparkContext.parallelize(data,4)
print(rdd.glom().collect())

print("After apply map function ")
rdd1 = rdd.map(lambda x : str(x))
rdd1.glom().collect()

[[9, 25, 4], [40, 24, 16], [49, 11, 31], [35, 3, 35, 15, 35, 50]]
After apply map function 


[['9', '25', '4'],
 ['40', '24', '16'],
 ['49', '11', '31'],
 ['35', '3', '35', '15', '35', '50']]

#2.filter()
A .filter() transformation is an operation in PySpark for filtering elements from a PySpark RDD. The .filter() transformation takes in an anonymous function with a condition. Again, since it’s a transformation, it returns an RDD having elements that had passed the given condition.

In [36]:
#filter() it return the partition of elements which satisfyes the condition.

d = spark.sparkContext.parallelize([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],4)
print(d.glom().collect())

D = d.filter(lambda x: x%2==0)
print(D.glom().collect())



[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12, 13, 14, 15]]
[[2], [4, 6], [8], [10, 12, 14]]


# 3.Union()

The .union() transformation combines two RDDs and returns the union of the input two RDDs

In [39]:
rr = rdd.union(RDD)
RDD.count(),rdd.count(),rr.count()


(100, 15, 115)

# 4.flatMap()
The .flatMap() transformation peforms same as the .map() transformation except the fact that .flatMap() transformation return seperate values for each element from original RDD

In [41]:
names = ['shiva','shnakar','Hari','krishna','samba','ravi','bharath','ajay','ragava','munna']
name_rdd = spark.sparkContext.parallelize(names,2)


new_rdd = name_rdd.flatMap(lambda x:(x.upper(),x.lower()))
new_rdd.glom().collect()

[['SHIVA',
  'shiva',
  'SHNAKAR',
  'shnakar',
  'HARI',
  'hari',
  'KRISHNA',
  'krishna',
  'SAMBA',
  'samba'],
 ['RAVI',
  'ravi',
  'BHARATH',
  'bharath',
  'AJAY',
  'ajay',
  'RAGAVA',
  'ragava',
  'MUNNA',
  'munna']]

#RDD ACTIONS:
Actions are a kind of operation which are applied on an RDD to produce a single value. These methods are applied on a resultant RDD and produces a non-RDD value, thus removing the laziness of the transformation of RDD.

# 1.collect()
The .collect() action on an RDD returns a list of all the elements of the RDD. It’s a great asset for displaying all the contents of our RDD.

In [17]:
import numpy as np
np.random.seed(100)
data = np.random.randint(1,1000,size=100)
data

array([521, 793, 836, 872, 856,  80, 945, 907, 351, 949, 867,  54, 579,
       739, 527, 803, 753, 281, 656, 229, 876, 317, 571, 913, 508, 650,
        94,  87, 387, 668, 877, 901, 416, 898, 142, 758, 724, 613,   5,
       604, 956, 836, 136,  50, 432, 706, 318, 783, 696, 968, 764, 337,
         3, 890, 618, 479, 404, 995,  64, 182, 284, 825, 239, 370, 927,
       945, 304, 680, 878, 807, 173, 275, 193, 953, 931, 438, 715, 274,
       585, 526, 619,  31,  18,  54,  69, 947, 489, 348, 476, 980, 694,
       847,   1,  14, 186, 461, 363, 132, 583, 644])

In [19]:
RDD = spark.sparkContext.parallelize(data,10)
RDD.glom().collect()

[[521, 793, 836, 872, 856, 80, 945, 907, 351, 949],
 [867, 54, 579, 739, 527, 803, 753, 281, 656, 229],
 [876, 317, 571, 913, 508, 650, 94, 87, 387, 668],
 [877, 901, 416, 898, 142, 758, 724, 613, 5, 604],
 [956, 836, 136, 50, 432, 706, 318, 783, 696, 968],
 [764, 337, 3, 890, 618, 479, 404, 995, 64, 182],
 [284, 825, 239, 370, 927, 945, 304, 680, 878, 807],
 [173, 275, 193, 953, 931, 438, 715, 274, 585, 526],
 [619, 31, 18, 54, 69, 947, 489, 348, 476, 980],
 [694, 847, 1, 14, 186, 461, 363, 132, 583, 644]]

# 2.Count()
The count() action on an RDD is an operation that returns the number of elements of our RDD.

In [18]:
RDD.count()

100

# 3.first()

The .first() action on an RDD returns the first element from our RDD.

In [21]:
RDD.first()


521

# 4.take(n)
The .take(n) action on an RDD returns n number of elements from the RDD. The ‘n’ argument takes an integer which refers to the number of elements we want to extract from the RDD.

In [26]:
RDD.take(2)

[521, 793]

# 5.reduce()
The .reduce() Action takes two elements from the given RDD and operates. This operation is performed using an anonymous function or lambda.
This is a costly opearation as it is iterates through every element of RDD.

In [27]:
RDD.reduce(lambda x,y:x+y)
#Sum of all the elements

53502

# 6.SaveAsTextFile()
The .saveAsTextFile() Action is used to save the resultant RDD as a text file. We can also specify the path to which file needed to be saved.

In [30]:
RDD.coalesce(1)
RDD.saveAsTextFile('sample_RDD1.txt')

# PySpark Pair RDD Operations

PySpark has a dedicated set of operations for Pair RDDs. Pair RDDs are a special kind of data structure in PySpark in the form of key-value pairs, and that’s how it got its name. Practically, the Pair RDDs are used more widely because of the reason that most of the real-world data is in the form of Key/Value pairs. The Pair RDDs use different terminology for key and value. The key is known as the identifier while the value is known as data.

# Transformations in Pair RDDs

#1.reduceByKey()

The .reduceByKey() transformation performs multiple parallel processes for each key in the data and combines the values for the same keys. It uses an anonymous function or lambda to perform the task. Since it’s a transformation, it returns an RDD as a result.

In [45]:
marks= [('shiva',100),('rohit',264),('virat',89),('virat',109),('shiva',400),('rohit',100)]

marks_rdd = spark.sparkContext.parallelize(marks)

print(marks_rdd.glom().collect())


new_marks_rdd = marks_rdd.reduceByKey(lambda x,y: x+y)

new_marks_rdd.glom().collect()

[[('shiva', 100), ('rohit', 264), ('virat', 89)], [('virat', 109), ('shiva', 400), ('rohit', 100)]]


[[('shiva', 500), ('virat', 198)], [('rohit', 364)]]

#2.sortByKey()

The .sortByKey() transformation sorts the input data by keys from key-value pairs either in ascending or descending order. It returns a unique RDD as a result.

In [48]:
sorted_marks_rdd = marks_rdd.sortByKey()
sorted_marks_rdd.collect()

[('rohit', 264),
 ('rohit', 100),
 ('shiva', 100),
 ('shiva', 400),
 ('virat', 89),
 ('virat', 109)]

# 3.groupByKey()
The .groupByKey() transformation groups all the values in the given data with the same key together.

In [52]:
grouped_marks_rdd = marks_rdd.groupByKey().collect()

for key,value in grouped_marks_rdd:
  print(key,list(value))


#groupByKey() transformation on the marks_rdd.
#Then we used the .collect() action to get the results and saved the results to grouped_marks_rdd.
#Since grouped_marks_rdd is a dictionary item type, we applied the for loop on grouped_marks_rdd  to get a list of marks for each student in each line.
#We also added list() to the values .

shiva [100, 400]
virat [89, 109]
rohit [264, 100]


# Actions on Pair RDDs

# 1.countByKey()

The .countByKey() option is used to count the number of values for each key in the given data. This action returns a dictionary and one can extract the keys and values by iterating over the extracted dictionary using loops. Since we are getting a dictionary as a result, we can also use the dictionary methods such as .keys(), .values() and .items().

In [54]:
r = marks_rdd.countByKey()
for key,value in r.items():
  print(key,value)

shiva 2
rohit 2
virat 2


# RDD types

PairRDDFunctions or PairRDD : Pair RDD is a key-value

ShuffledRDD

DoubleRDD

SequenceFileRDD

HadoopRDD

ParallelCollectionRDD

#shuffeled operations
Shuffling is a mechanism PySpark uses to redistribute the data across different executors and even across machines. PySpark shuffling triggers when we perform certain transformation operations like gropByKey(), reduceByKey(), join() on RDDS.

PySpark Shuffle is an expensive operation since it involves the following

Disk I/O

Involves data serialization and deserialization

Network I/O

# Pyspark Cache and persist

PySpark Cache and Persist are optimization techniques to improve the performance of the RDD jobs that are iterative and interactive.

Though PySpark provides computation 100 x times faster than traditional Map Reduce jobs, If you have not designed the jobs to reuse the repeating computations you will see degrade in performance when you are dealing with billions or trillions of data. Hence, we need to look at the computations and use optimization techniques as one of the ways to improve performance.

Using cache() and persist() methods, PySpark provides an optimization mechanism to store the intermediate computation of an RDD so they can be reused in subsequent actions.


When you persist or cache an RDD, each worker node stores it’s partitioned data in memory or disk and reuses them in other actions on that RDD. And Spark’s persisted data on nodes are fault-tolerant meaning if any partition is lost, it will automatically be recomputed using the original transformations that created it.

# Advantages of Persisting RDD

Cost efficient – PySpark computations are very expensive hence reusing the computations are used to save cost.

Time efficient – Reusing the repeated computations saves lots of time.

Execution time – Saves execution time of the job which allows us to perform more jobs on the same cluster.

## RDD Cache
PySpark RDD cache() method by default saves RDD computation to storage level `MEMORY_ONLY` meaning it will store the data in the JVM heap as unserialized objects.

PySpark cache() method in RDD class internally calls persist() method which in turn uses sparkSession.sharedState.cacheManager.cacheQuery to cache the result set of RDD.

# RDD Persist
PySpark persist() method is used to store the RDD to one of the storage levels MEMORY_ONLY,MEMORY_AND_DISK, MEMORY_ONLY_SER, MEMORY_AND_DISK_SER, DISK_ONLY, MEMORY_ONLY_2,MEMORY_AND_DISK_2 and more.

PySpark persist has two signature first signature doesn’t take any argument which by default saves it to <strong>MEMORY_ONLY</strong> storage level and the second signature which takes StorageLevel as an argument to store it to different storage levels.

# RDD Unpersist
PySpark automatically monitors every persist() and cache() calls you make and it checks usage on each node and drops persisted data if not used or by using least-recently-used (LRU) algorithm. You can also manually remove using unpersist() method. unpersist() marks the RDD as non-persistent, and remove all blocks for it from memory and disk.



# Persistence Storage Levels
All different storage level PySpark supports are available at org.apache.spark.storage.StorageLevel class. Storage Level defines how and where to store the RDD.

**MEMORY_ONLY **– This is the default behavior of the RDD cache() method and stores the RDD as deserialized objects to JVM memory. When there is no enough memory available it will not save to RDD of some partitions and these will be re-computed as and when required. This takes more storage but runs faster as it takes few CPU cycles to read from memory.

**MEMORY_ONLY_SER –** This is the same as MEMORY_ONLY but the difference being it stores RDD as serialized objects to JVM memory. It takes lesser memory (space-efficient) then MEMORY_ONLY as it saves objects as serialized and takes an additional few more CPU cycles in order to deserialize.

**MEMORY_ONLY_2 –** Same as MEMORY_ONLY storage level but replicate each partition to two cluster nodes.

**MEMORY_ONLY_SER_2 –** Same as MEMORY_ONLY_SER storage level but replicate each partition to two cluster nodes.

**MEMORY_AND_DISK –**In this Storage Level, The RDD will be stored in JVM memory as a deserialized objects. When required storage is greater than available memory, it stores some of the excess partitions in to disk and reads the data from disk when it required. It is slower as there is I/O involved.

**MEMORY_AND_DISK_SER –** This is same as MEMORY_AND_DISK storage level difference being it serializes the RDD objects in memory and on disk when space not available.

**MEMORY_AND_DISK_2 –** Same as MEMORY_AND_DISK storage level but replicate each partition to two cluster nodes.

**MEMORY_AND_DISK_SER_2 –** Same as MEMORY_AND_DISK_SER storage level but replicate each partition to two cluster nodes.

**DISK_ONLY –** In this storage level, RDD is stored only on disk and the CPU computation time is high as I/O involved.

**DISK_ONLY_2 –** Same as DISK_ONLY storage level but replicate each partition to two cluster nodes.

# PySpark Shared Variables :

Different types of PySpark Shared variables and how they are used in PySpark transformations.

When PySpark executes transformation using map() or reduce() operations, It executes the transformations on a remote node by using the variables that are shipped with the tasks and these variables are not sent back to PySpark Driver hence there is no capability to reuse and sharing the variables across tasks. PySpark shared variables solve this problem using the below two techniques. PySpark provides two types of shared variables.

**Broadcast variables** (read-only shared variable)

**Accumulator variables** (updatable shared variables)

**Broadcast read-only Variables**
Broadcast variables are read-only shared variables that are cached and available on all nodes in a cluster in-order to access or use by the tasks. Instead of sending this data along with every task, PySpark distributes broadcast variables to the machine using efficient broadcast algorithms to reduce communication costs.

One of the best use-case of PySpark RDD Broadcast is to use with lookup data for example zip code, state, country lookups e.t.c

When you run a PySpark RDD job that has the Broadcast variables defined and used, PySpark does the following.

PySpark breaks the job into stages that have distributed shuffling and actions are executed with in the stage.
Later Stages are also broken into tasks
PySpark broadcasts the common data (reusable) needed by tasks within each stage.
The broadcasted data is cache in serialized format and deserialized before executing each task.
The PySpark Broadcast is created using the broadcast(v) method of the SparkContext class. This method takes the argument v that you want to broadcast.


broadcastVar = sc.broadcast([0, 1, 2, 3])

broadcastVar.value

Note that broadcast variables are not sent to executors with sc.broadcast(variable) call instead, they will be sent to executors when they are first used.

Refer to PySpark RDD Broadcast shared variable for more detailed example.

# Accumulators

PySpark Accumulators are another type shared variable that are only “added” through an associative and commutative operation and are used to perform counters (Similar to Map-reduce counters) or sum operations.

PySpark by default supports creating an accumulator of any numeric type and provides the capability to add custom accumulator types. Programmers can create following accumulators

**named accumulators**

**unnamed accumulators**

When you create a named accumulator, you can see them on PySpark web UI under the “Accumulator” tab. On this tab, you will see two tables; the first table “accumulable” – consists of all named accumulator variables and their values. And on the second table “Tasks” – value for each accumulator modified by a task.

Where as unnamed accumulators are not shows on PySpark web UI, For all practical purposes it is suggestable to use named accumulators.

Accumulator variables are created using SparkContext.longAccumulator(v)

PySpark by default provides accumulator methods for long, double and collection types. All these methods are present in SparkContext class and return LongAccumulator, DoubleAccumulator, and CollectionAccumulator respectively.

Long Accumulator

Double Accumulator

Collection Accumulator